In [1]:
import random

from tqdm import tqdm as tq
import numpy as np
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
def get_sample():
    start = random.randint(0,100)
    nos = 100
    step = .05
    samples = torch.arange(start=start, end=start+nos*step, step=step, device=device)
    sins = torch.sin(samples)
    
    X = sins[:50]
    Y = sins[50:]
    
    return X.view(-1,1,1,1),Y

In [3]:
class SineEncoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(SineEncoder, self).__init__()
        self.input_size=input_size
        self.hidden_size=hidden_size
        self.lstm = nn.LSTM(input_size=input_size,
                    hidden_size=hidden_size,
                    num_layers=1,
                    batch_first=False,
                    bidirectional=False)
        
    def init_hidden(self):
        return (torch.zeros(1, 1, self.hidden_size, device=device), 
                torch.zeros(1, 1, self.hidden_size, device=device))
        
    def forward(self, input_tensor, hidden_tensor):
        output_tensor, hidden_tensor = self.lstm(input_tensor, hidden_tensor)
        return hidden_tensor

In [4]:
class SineDecoder(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SineDecoder, self).__init__()
        self.input_size=input_size
        self.hidden_size=hidden_size
        self.output_size=output_size
        self.lstm = nn.LSTM(input_size=input_size,
                    hidden_size=hidden_size,
                    num_layers=1,
                    batch_first=False,
                    bidirectional=False)
        self.linear = nn.Linear(hidden_size, output_size)
        
    def forward(self, input_tensor, hidden_tensor):
        output_tensor, hidden_tensor = self.lstm(input_tensor, hidden_tensor)
        output_tensor = self.linear(output_tensor)
        output_tensor = torch.tanh(output_tensor)
        return output_tensor, hidden_tensor

In [5]:
encoder = SineEncoder(1, 128).to(device)
decoder = SineDecoder(1, 128, 1).to(device)

In [6]:
optim_encoder = torch.optim.SGD(encoder.parameters(), lr=.005)
optim_decoder = torch.optim.SGD(decoder.parameters(), lr=.005)

In [7]:
# def set_lr(opt, lr):
#     for p in opt.param_groups:
#         p['lr'] = lr
# set_lr(optim_encoder, .001)
# set_lr(optim_decoder, .001)

In [8]:
def train(n):
    losses = 0
    kks = 0
    for itr in range(1, n+1):
        X,Y = get_sample()
        hidden_tensor = encoder.init_hidden()

        for x in X:
            hidden_tensor = encoder.forward(x, hidden_tensor)

        output_tensor = X[-1]
        loss = 0
        kk = 0
        for i in range(Y.shape[0]):
            output_tensor, hidden_tensor = decoder.forward(output_tensor, hidden_tensor)
            loss += (output_tensor.view(-1)[0] - Y[i]) ** 2
            kk += torch.abs(output_tensor.view(-1)[0] - Y[i]).item()

        losses += loss.item()
        kks += kk
        
        optim_encoder.zero_grad()
        optim_decoder.zero_grad()

        loss.backward()

        optim_encoder.step()
        optim_decoder.step()
        
        if itr % (n/25) == 0:
            print(100*itr/n, losses/itr, kks/itr)

In [9]:
train(400000)

4.0 5.843283358820525 8.694241497620396
8.0 2.929315435873912 4.65844215982731
12.0 1.9558709118421806 3.265450138480567
16.0 1.4685231856227274 2.5492392896043503
20.0 1.1757191536575382 2.1083688572443005
24.0 0.9804737159302446 1.8104333394659538
28.0 0.8409505330405012 1.5926638717879225
32.0 0.7361425199270911 1.4251363012052798
36.0 0.654581680380998 1.293130293218847
40.0 0.5893225116049766 1.1865797109831995
44.0 0.5359234580871958 1.0989155403677486
48.0 0.49141410475061253 1.0250526325290006
52.0 0.45371241130125894 0.9606056301825058
56.0 0.42140259056607043 0.9054154879331768
60.0 0.39339053772887517 0.8569704708742769
64.0 0.36887031913348467 0.8138685544220865
68.0 0.34724033849783925 0.7758951682566407
72.0 0.3280021112671306 0.7415126340516702
76.0 0.3107936759662068 0.7107517669124146
80.0 0.2952947163799454 0.6825189985696388
84.0 0.28127232184505324 0.6568662611291404
88.0 0.2685219941328346 0.6334197276459242
92.0 0.2568775852627975 0.611658374201275
96.0 0.24620375

In [10]:
encoder.eval()
decoder.eval()

SineDecoder(
  (lstm): LSTM(1, 128)
  (linear): Linear(in_features=128, out_features=1, bias=True)
)

In [11]:
def test():
    X,Y = get_sample()
    hidden_tensor = encoder.init_hidden()

    for x in X:
        hidden_tensor = encoder.forward(x, hidden_tensor)
    
    kk = 0
    Y_pred = []
    output_tensor = X[-1]
    for i in range(Y.shape[0]):
        output_tensor, hidden_tensor = decoder.forward(output_tensor, hidden_tensor)
        Y_pred.append(output_tensor.view(-1)[0].item())
        
    print(Y.cpu().detach().numpy())
    print(np.array(Y_pred))
    print(kk)

In [12]:
test()

[ 0.9284657   0.90874255  0.886748    0.86253524  0.83616847  0.80771166
  0.77723616  0.744818    0.71053547  0.67447966  0.636738    0.5974049
  0.55657864  0.51435804  0.47085497  0.42617506  0.38042995  0.33373404
  0.28620028  0.23795484  0.18911463  0.13980174  0.09013943  0.04024802
 -0.00974019 -0.05970405 -0.10951869 -0.1590596  -0.20820668 -0.25682962
 -0.30481064 -0.3520298  -0.39836907 -0.4437161  -0.48795062 -0.53096557
 -0.57265335 -0.6129099  -0.65163743 -0.6887332  -0.7241076  -0.7576722
 -0.789343   -0.8190431  -0.84669375 -0.87222815 -0.8955825  -0.91669846
 -0.9355246  -0.9520109 ]
[ 0.93916261  0.90958005  0.88029206  0.85978246  0.83635867  0.80938393
  0.77901059  0.7460829   0.711923    0.67550194  0.63681686  0.59741062
  0.55722725  0.51589042  0.4732514   0.4292033   0.38374555  0.33701351
  0.28922302  0.24061649  0.19142976  0.14188443  0.09216898  0.04241822
 -0.0073105  -0.05702263 -0.10674248 -0.15643345 -0.20594265 -0.25500301
 -0.3033092  -0.35061246 -0

In [13]:
checkpoint = {'optim_encoder' : optim_encoder, 'optim_decoder' : optim_decoder, 'encoder_state_dict': encoder.state_dict(),  'decoder_state_dict': decoder.state_dict()}
torch.save(checkpoint, "checkpoint.pth")

text_file = open("hudai.txt", "w")
text_file.write('hehe')
text_file.close()